<a href="https://colab.research.google.com/github/ammaster10s/Year3/blob/main/Evaluate_OCT_NV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
from pycocotools import mask as maskUtils
import json
from tensorflow import keras
from keras import backend as KK  # Import Keras backend
import tensorflow as tf

from keras import backend

# Load the pretrained U-Net model
# Load the model and pass K as a custom object
model_path = '/content/drive/MyDrive/Working/models/attention_unet_model_62epoch_423_4.h5'
unet_model = load_model(model_path)
# unet_model = load_model(model_path)

# Load and preprocess the full image for prediction
def load_full_image(image_path, target_size=(256, 256)):
    full_image = load_img(image_path, color_mode='grayscale', target_size=target_size)
    full_image_array = img_to_array(full_image) / 255.0
    full_image_array = np.expand_dims(full_image_array, axis=0)  # Add batch dimension
    return full_image_array

# Function to handle both RLE and polygon segmentation formats in COCO
def segmentation_to_mask(segmentation, width, height):
    if isinstance(segmentation, dict):  # RLE format (compressed RLE)
        # print("Decoding RLE mask")
        rle = maskUtils.frPyObjects(segmentation, height, width)
        mask = maskUtils.decode(rle)
        return mask
    elif isinstance(segmentation, list):  # Polygon format
        # print("Processing polygon mask")
        mask = np.zeros((height, width), dtype=np.uint8)
        # Create a binary mask from polygons
        for polygon in segmentation:
            poly = np.array(polygon).reshape((-1, 2))
            img = Image.new('L', (width, height), 0)
            ImageDraw.Draw(img).polygon([tuple(p) for p in poly], outline=1, fill=1)
            mask = np.array(img, dtype=np.uint8)
        return mask
    else:
        raise TypeError("Unsupported segmentation format")

# Function to parse the COCO JSON file and return annotations
def parse_coco_json(json_file, image_dir):
    with open(json_file, 'r') as f:
        coco_data = json.load(f)

    annotations = []
    image_data = {}

    # Map image ID to file name and other metadata
    for img_info in coco_data['images']:
        image_data[img_info['id']] = {
            'file_name': img_info['file_name'],
            'width': img_info['width'],
            'height': img_info['height']
        }

    # Extract segmentation and mask information
    for ann in coco_data['annotations']:
        image_id = ann['image_id']
        image_info = image_data[image_id]
        image_filename = image_info['file_name']
        width = image_info['width']
        height = image_info['height']

        # Get the segmentation mask (COCO format supports RLE or polygons)
        mask = segmentation_to_mask(ann['segmentation'], width, height)

        annotations.append((image_filename, mask))

    return annotations

# Function to compute IoU (Intersection over Union)
def compute_iou(predicted_mask, ground_truth_mask):
    # Flatten masks to compute IoU on 1D arrays
    predicted_mask = predicted_mask.flatten()
    ground_truth_mask = ground_truth_mask.flatten()

    # Compute IoU
    intersection = np.sum(np.logical_and(predicted_mask, ground_truth_mask))
    union = np.sum(np.logical_or(predicted_mask, ground_truth_mask))
    iou = intersection / union if union != 0 else 0

    return iou

# Function to compute Dice coefficient
def compute_dice(predicted_mask, ground_truth_mask):
    # Flatten masks
    predicted_mask = predicted_mask.flatten()
    ground_truth_mask = ground_truth_mask.flatten()

    # Compute Dice Coefficient
    intersection = np.sum(np.logical_and(predicted_mask, ground_truth_mask))
    dice = (2. * intersection) / (np.sum(predicted_mask) + np.sum(ground_truth_mask))

    return dice

# Function to evaluate the model on each image and mask from COCO annotations
def evaluate_model_on_coco(test_dir, coco_json_file, threshold=0.5):
    iou_scores = []
    dice_scores = []
    annotations = parse_coco_json(coco_json_file, test_dir)
    counter = 0

    # Loop through all images and masks from COCO annotations
    for image_filename, ground_truth_mask in annotations:
        full_image_path = os.path.join(test_dir, image_filename)

        if not os.path.exists(full_image_path):
            # print(f"Image {image_filename} not found, skipping...")
            continue

        # Load and preprocess the image
        full_image = load_full_image(full_image_path)

        # Predict the mask for the full image
        predicted_mask = unet_model.predict(full_image)[0, :, :, 0]  # Extract the 2D mask

        # Apply threshold to convert the predicted mask to binary
        predicted_mask_binary = (predicted_mask > threshold).astype(np.uint8)

        # Resize the ground truth mask to match the predicted mask size
        ground_truth_mask_resized = np.array(Image.fromarray(ground_truth_mask).resize((256, 256), Image.NEAREST))

        # Calculate IoU and Dice scores
        iou = compute_iou(predicted_mask_binary, ground_truth_mask_resized)
        dice = compute_dice(predicted_mask_binary, ground_truth_mask_resized)

        iou_scores.append(iou)
        dice_scores.append(dice)

        # Display evaluation results
        print(f"{image_filename}: IoU = {iou}, Dice Coefficient = {dice}")

        # Optionally plot every 10th result
        # Optionally plot every 10th result
        if counter % 10 == 0:
            plt.figure(figsize=(16, 8))  # Adjusting the figure size for four subplots

            # Subplot 1: Display the original full image
            plt.subplot(1, 4, 1)
            plt.imshow(np.squeeze(full_image), cmap='gray')
            plt.title(f'Full Image: {image_filename}')

            # Subplot 2: Display the predicted NV mask heatmap
            plt.subplot(1, 4, 2)
            plt.imshow(predicted_mask, cmap='jet', alpha=0.6)
            plt.title('Predicted NV Heatmap')
            plt.colorbar()

            # Subplot 3: Overlay predicted mask on the full image
            plt.subplot(1, 4, 3)
            plt.imshow(np.squeeze(full_image), cmap='gray')  # Original image
            plt.imshow(predicted_mask_binary, cmap='jet', alpha=0.5)  # Predicted mask overlay
            plt.title('Overlay: Full Image with Predicted Mask')

            # Subplot 4: Display the ground truth mask
            plt.subplot(1, 4, 4)
            plt.imshow(ground_truth_mask_resized, cmap='gray')
            plt.title('Ground Truth Mask')

            # Show the plots
            plt.show()


        counter += 1

    # Print average metrics
    print(f"Average IoU: {np.mean(iou_scores)}")
    print(f"Average Dice Coefficient: {np.mean(dice_scores)}")

# Paths to your dataset (Images and COCO annotations)
test_image_dir = '/content/drive/MyDrive/Working/train/CNV'
test_json_annotation_file = '/content/drive/MyDrive/Working/train/instances_default.json'  # COCO JSON file with annotations

# Run the evaluation
evaluate_model_on_coco(test_image_dir, test_json_annotation_file)


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
from pycocotools import mask as maskUtils
import json
import tensorflow as tf # Import tensorflow


model_path = '/content/drive/MyDrive/Working/models/attention_unet_model_62epoch_423_4.h5'
unet_model = tf.keras.models.load_model(model_path)

# Load and preprocess the full image for prediction
def load_full_image(image_path, target_size=(256, 256)):
    full_image = load_img(image_path, color_mode='grayscale', target_size=target_size)
    full_image_array = img_to_array(full_image) / 255.0
    full_image_array = np.expand_dims(full_image_array, axis=0)  # Add batch dimension
    return full_image_array

# Function to handle both RLE and polygon segmentation formats in COCO
def segmentation_to_mask(segmentation, width, height):
    if isinstance(segmentation, dict):  # RLE format (compressed RLE)
        print("Decoding RLE mask")
        rle = maskUtils.frPyObjects(segmentation, height, width)
        mask = maskUtils.decode(rle)
        return mask
    elif isinstance(segmentation, list):  # Polygon format
        print("Processing polygon mask")
        mask = np.zeros((height, width), dtype=np.uint8)
        # Create a binary mask from polygons
        for polygon in segmentation:
            poly = np.array(polygon).reshape((-1, 2))
            img = Image.new('L', (width, height), 0)
            ImageDraw.Draw(img).polygon([tuple(p) for p in poly], outline=1, fill=1)
            mask = np.array(img, dtype=np.uint8)
        return mask
    else:
        raise TypeError("Unsupported segmentation format")

# Function to parse the COCO JSON file and return annotations
def parse_coco_json(json_file, image_dir):
    with open(json_file, 'r') as f:
        coco_data = json.load(f)

    annotations = []
    image_data = {}

    # Map image ID to file name and other metadata
    for img_info in coco_data['images']:
        image_data[img_info['id']] = {
            'file_name': img_info['file_name'],
            'width': img_info['width'],
            'height': img_info['height']
        }

    # Extract segmentation and mask information
    for ann in coco_data['annotations']:
        image_id = ann['image_id']
        image_info = image_data[image_id]
        image_filename = image_info['file_name']
        width = image_info['width']
        height = image_info['height']

        # Get the segmentation mask (COCO format supports RLE or polygons)
        mask = segmentation_to_mask(ann['segmentation'], width, height)

        annotations.append((image_filename, mask))

    return annotations

# Function to evaluate if the predicted mask correctly matches the ground truth
def evaluate_prediction_accuracy(predicted_mask, ground_truth_mask, threshold=0.5):
    # Apply threshold to predicted mask
    predicted_mask_binary = (predicted_mask > threshold).astype(np.uint8)

    # Check if predicted mask overlaps with ground truth (1 where predicted, 1 where ground truth)
    # Correct pixels = intersection of predicted and ground truth
    correct_pixels = np.sum(np.logical_and(predicted_mask_binary, ground_truth_mask))

    # Return 1 if there are any correct pixels, else 0
    return 1 if correct_pixels > 0 else 0

# Function to evaluate the model on each image and mask from COCO annotations
def evaluate_model_on_coco(test_dir, coco_json_file, threshold=0.5):
    total_images = 0
    total_correct = 0
    annotations = parse_coco_json(coco_json_file, test_dir)

    # Loop through all images and masks from COCO annotations
    for image_filename, ground_truth_mask in annotations:
        full_image_path = os.path.join(test_dir, image_filename)

        if not os.path.exists(full_image_path):
            print(f"Image {image_filename} not found, skipping...")
            continue

        # Load and preprocess the image
        full_image = load_full_image(full_image_path)

        # Predict the mask for the full image
        predicted_mask = unet_model.predict(full_image)[0, :, :, 0]  # Extract the 2D mask

        # predicted_mask = model.predict(full_image)[0, :, :, 0]  # Extract the 2D mask

        # Resize the ground truth mask to match the predicted mask size
        ground_truth_mask_resized = np.array(Image.fromarray(ground_truth_mask).resize((256, 256), Image.NEAREST))

        # Evaluate the prediction correctness
        is_correct = evaluate_prediction_accuracy(predicted_mask, ground_truth_mask_resized, threshold)

        total_images += 1
        total_correct += is_correct

        # Display evaluation results
        print(f"{image_filename}: Prediction Correct: {is_correct}")

    # Print final accuracy
    accuracy = total_correct / total_images if total_images > 0 else 0
    print(f"Total Correct: {total_correct}/{total_images}")
    print(f"Accuracy: {accuracy * 100:.2f}%")

# Paths to your dataset (Images and COCO annotations)
test_image_dir = '/content/drive/MyDrive/Working/val/CNV35'
test_json_annotation_file = '/content/drive/MyDrive/Working/val/instances_default.json'  # COCO JSON file with annotations

# Run the evaluation
evaluate_model_on_coco(test_image_dir, test_json_annotation_file)


In [ ]:
import os
import random
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from tensorflow.keras.models import load_model

# Load and preprocess the full image for prediction
def load_full_image(image_path, target_size=(256, 256)):
    full_image = load_img(image_path, color_mode='grayscale', target_size=target_size)
    full_image_array = img_to_array(full_image) / 255.0
    full_image_array = np.expand_dims(full_image_array, axis=0)
    return full_image_array

# Load your model
loaded_model = load_model('/content/drive/MyDrive/Working/models/attention_unet_model_62epoch_423_4.h5')

# Directory path containing the images to test
test_dir = '/content/drive/MyDrive/Working/Test/Mix'

# Get list of image files and shuffle them randomly
image_files = [f for f in os.listdir(test_dir) if f.endswith(('.jpeg', '.jpg', '.png'))]
random.shuffle(image_files)  # Shuffle the list randomly

# Loop through the shuffled image files and display results
for image_filename in image_files:
    full_image_path = os.path.join(test_dir, image_filename)

    # Load and preprocess the image
    full_image = load_full_image(full_image_path)

    # Predict the mask for the full image
    predicted_mask = loaded_model.predict(full_image)[0, :, :, 0]  # Extract the 2D mask

    # Apply threshold to get a binary mask
    threshold = 0.5
    predicted_mask_binary = (predicted_mask > threshold).astype(np.uint8)

    # Plot the results for each image
    plt.figure(figsize=(12, 8))

    # Display the original full image
    plt.subplot(1, 3, 1)
    plt.imshow(np.squeeze(full_image), cmap='gray')
    plt.title(f'Full Image: {image_filename}')

    # Display the predicted NV mask heatmap
    plt.subplot(1, 3, 2)
    plt.imshow(predicted_mask, cmap='jet', alpha=0.6)
    plt.title('Predicted NV Heatmap')
    plt.colorbar()

    # Overlay predicted mask on the full image
    plt.subplot(1, 3, 3)
    plt.imshow(np.squeeze(full_image), cmap='gray')  # Original image
    plt.imshow(predicted_mask_binary, cmap='jet', alpha=0.5)  # Predicted mask overlay
    plt.title('Overlay: Full Image with Predicted Mask')

    # Show the plots
    plt.show()


In [ ]:
import os
import random
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from tensorflow.keras.models import load_model
from matplotlib.backends.backend_pdf import PdfPages

# Load and preprocess the full image for prediction
def load_full_image(image_path, target_size=(256, 256)):
    full_image = load_img(image_path, color_mode='grayscale', target_size=target_size)
    full_image_array = img_to_array(full_image) / 255.0
    full_image_array = np.expand_dims(full_image_array, axis=0)
    return full_image_array

# Load your model
loaded_model = load_model('/content/drive/MyDrive/Working/models/attention_unet_model_62epoch_423_4.h5')

# Directory path containing the images to test
test_dir = '/content/drive/MyDrive/Working/Test/Mix'

# Get list of image files and shuffle them randomly
image_files = [f for f in os.listdir(test_dir) if f.endswith(('.jpeg', '.jpg', '.png'))]
random.shuffle(image_files)  # Shuffle the list randomly

# Define the output PDF file
output_pdf_path = '/content/drive/MyDrive/Working/output_results.pdf'

# Create a PdfPages object to save multiple pages to a single PDF
with PdfPages(output_pdf_path) as pdf:
    # Loop through the shuffled image files
    for image_filename in image_files:
        full_image_path = os.path.join(test_dir, image_filename)

        # Load and preprocess the image
        full_image = load_full_image(full_image_path)

        # Predict the mask for the full image
        predicted_mask = loaded_model.predict(full_image)[0, :, :, 0]  # Extract the 2D mask

        # Apply threshold to get a binary mask
        threshold = 0.5
        predicted_mask_binary = (predicted_mask > threshold).astype(np.uint8)

        # Plot the results for each image
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))

        # Display the original full image
        axes[0].imshow(np.squeeze(full_image), cmap='gray')
        axes[0].set_title(f'Full Image: {image_filename}')
        axes[0].axis('off')

        # Display the predicted NV mask heatmap
        im = axes[1].imshow(predicted_mask, cmap='jet', alpha=0.6)
        axes[1].set_title('Predicted NV Heatmap')
        axes[1].axis('off')
        plt.colorbar(im, ax=axes[1], fraction=0.046, pad=0.04)

        # Overlay predicted mask on the full image
        axes[2].imshow(np.squeeze(full_image), cmap='gray')  # Original image
        axes[2].imshow(predicted_mask_binary, cmap='jet', alpha=0.5)  # Predicted mask overlay
        axes[2].set_title('Overlay: Full Image with Predicted Mask')
        axes[2].axis('off')

        # Save the current figure to the PDF
        pdf.savefig(fig)
        plt.close(fig)  # Close the figure to save memory

print(f"Results saved to {output_pdf_path}")
